<a href="https://colab.research.google.com/github/lc0/practical-ml-dahoam/blob/part-1/04_Production_use_tfjs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Production use
## TensorFlow js

In [2]:
!ls -laF

total 3352
drwxr-xr-x 1 root root    4096 May 23 13:50 ./
drwxr-xr-x 1 root root    4096 May 23 12:56 ../
-rw-r--r-- 1 root root 3384133 May 23 13:01 00-210.tfrec
-rw-r--r-- 1 root root    2547 May 23 12:56 adc.json
drwxr-xr-x 1 root root    4096 May 23 12:56 .config/
drwxr-xr-x 3 root root    4096 May 23 13:19 dataset/
drwx------ 8 root root    4096 May 23 12:58 google_dataset/
drwxr-xr-x 6 root root    4096 May 23 13:48 logs/
drwxr-xr-x 3 root root    4096 May 23 13:59 models/
drwxr-xr-x 1 root root    4096 May 15 16:23 sample_data/


In [4]:
!pip install tensorflowjs

     |████████████████████████████████| 13.9MB 7.7MB/s 
     |████████████████████████████████| 81kB 26.5MB/s 
     |████████████████████████████████| 86.8MB 53.0MB/s 
ERROR: thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
ERROR: s3fs 0.2.1 has requirement six>=1.12.0, but you'll have six 1.11.0 which is incompatible.
ERROR: jupyter-console 6.0.0 has requirement prompt-toolkit<2.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.16 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.11.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.3
    Uninstalling numpy-1.16.3:
      Successfully uninstalled numpy-1.16.3
  Found existing installation: six 1

Convert model

In [2]:
!tensorflowjs_converter --input_format keras \
                        models/model_transfered_mobilenet.h5 \
                        prod/tfjs

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'


In [3]:
!pip freeze | grep numpy

msgpack-numpy==0.4.3.2
numpy==1.15.1


In [5]:
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3MB 2.7MB/s 
ERROR: thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
ERROR: tensorflowjs 1.1.2 has requirement numpy==1.15.1, but you'll have numpy 1.16.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.11.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.3
    Uninstalling numpy-1.16.3:
      Successfully uninstalled numpy-1.16.3


In [0]:
!tensorflowjs_converter --input_format keras \
                        models/model_transfered_mobilenet.h5 \
                        prod/tfjs

In [4]:
!ls -laF prod/tfjs/

total 9832
drwxr-xr-x 2 root root    4096 May 23 14:11 ./
drwxr-xr-x 3 root root    4096 May 23 14:11 ../
-rw-r--r-- 1 root root 4194304 May 23 14:14 group1-shard1of3.bin
-rw-r--r-- 1 root root 4194304 May 23 14:14 group1-shard2of3.bin
-rw-r--r-- 1 root root 1564948 May 23 14:14 group1-shard3of3.bin
-rw-r--r-- 1 root root  101874 May 23 14:14 model.json


## App base

In [11]:
%%writefile prod/tfjs/index.html


<html>
<head>
    <!-- Load the latest version of TensorFlow.js -->
    <script src="https://unpkg.com/@tensorflow/tfjs"></script>
    <script src="https://unpkg.com/@tensorflow-models/mobilenet"></script>
    <script src="https://unpkg.com/@tensorflow-models/knn-classifier"></script>

</head>
<body>
    <style>
        .pred-container {
            margin-bottom: 20px;
        }
        .pred-container > div {
            display: inline-block;
            margin-right: 20px;
            vertical-align: top;
        }
        .row {
            display: table-row;
        }
        .cell {
            display: table-cell;
            padding-right: 20px;
        }
        #file-container {
            margin-bottom: 20px;
        }
        </style>
    <img id="img" crossOrigin src="shoe.png" width=192 height=192/>
    <div id="predictions"></div>


    <!-- Load index.js after the content of the page -->
    <script src="main.js"></script>
</body>
</html>

Overwriting prod/tfjs/index.html


In [44]:
%%writefile prod/tfjs/main.js



IMAGE_SIZE = 192
CLASSES = ['converse chuck 70', 'Exaggerated Soles', 'nike air max 90 essentials', 'nike react', 'Tie Dye sneakers']

const predictionsElement = document.getElementById('predictions');

async function app() {
  console.log('Loading model..');

  // Load the model.
  const model = await tf.loadLayersModel('/model.json');
  console.log('Sucessfully loaded model');

  // Make a prediction through the model on our image.
  const imgEl = document.getElementById('img');


  const result = await classify(model, imgEl);
  console.log(result);
}


async function classify(model, imgElement) {
  console.log('Predicting...');

  // The first start time includes the time it takes to extract the image
  // from the HTML and preprocess it, in additon to the predict() call.
  const startTime1 = performance.now();
  // The second start time excludes the extraction and preprocessing and
  // includes only the predict() call.
  let startTime2;
  const logits = tf.tidy(() => {
    // tf.browser.fromPixels() returns a Tensor from an image element.
    const img = tf.browser.fromPixels(imgElement, 3).toFloat();

    // Normalize the image from [0, 255] to [-1, 1].
    // const offset = tf.scalar(127.5);
    // const normalized = img.sub(offset).div(offset);
    normalized = img

    // Reshape to a single-element batch so we can pass it to predict.
    const batched = normalized.reshape([1, IMAGE_SIZE, IMAGE_SIZE, 3]);



    startTime2 = performance.now();
    // Make a prediction through mobilenet.
    return model.predict(batched);
  });

  console.log("Logits from model: ", logits)

  // Convert logits to probabilities and class names.
  classes = await getTopKClasses(logits, 2);
  console.log("Predicred classes: ", classes)

  const totalTime1 = performance.now() - startTime1;
  const totalTime2 = performance.now() - startTime2;
  console.log(`Done in ${Math.floor(totalTime1)} ms ` +
      `(not including preprocessing: ${Math.floor(totalTime2)} ms)`);

  showResults(imgElement, classes);
}

async function getTopKClasses(logits, topK) {
  const values = await logits.data();

  const valuesAndIndices = [];
  for (let i = 0; i < values.length; i++) {
    valuesAndIndices.push({value: values[i], index: i});
  }
  valuesAndIndices.sort((a, b) => {
    return b.value - a.value;
  });
  const topkValues = new Float32Array(topK);
  const topkIndices = new Int32Array(topK);
  for (let i = 0; i < topK; i++) {
    topkValues[i] = valuesAndIndices[i].value;
    topkIndices[i] = valuesAndIndices[i].index;
  }

  const topClassesAndProbs = [];
  for (let i = 0; i < topkIndices.length; i++) {
    topClassesAndProbs.push({
      className: CLASSES[topkIndices[i]],
      probability: topkValues[i]
    })
  }
  return topClassesAndProbs;
}

function showResults(imgElement, classes) {
  const predictionContainer = document.createElement('div');
  predictionContainer.className = 'pred-container';

  const imgContainer = document.createElement('div');
  imgContainer.appendChild(imgElement);
  predictionContainer.appendChild(imgContainer);

  const probsContainer = document.createElement('div');
  for (let i = 0; i < classes.length; i++) {
    const row = document.createElement('div');
    row.className = 'row';

    const classElement = document.createElement('div');
    classElement.className = 'cell';
    classElement.innerText = classes[i].className;
    row.appendChild(classElement);

    const probsElement = document.createElement('div');
    probsElement.className = 'cell';
    probsElement.innerText = classes[i].probability.toFixed(3);
    row.appendChild(probsElement);

    probsContainer.appendChild(row);
  }
  predictionContainer.appendChild(probsContainer);

  predictionsElement.insertBefore(
      predictionContainer, predictionsElement.firstChild);
}


app();

Overwriting prod/tfjs/main.js


Open port to serve remotely

In [0]:
python3 -m http.server

ngrock to open port to outside

In [13]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


--2019-05-23 15:22:42--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.236.200.27, 52.45.248.161, 52.22.236.254, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.236.200.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  7.19MB/s    in 2.2s    

2019-05-23 15:22:45 (7.19 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
get_ipython().system_raw('./ngrok http 8000 &')

In [50]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://5854a69d.ngrok.io


In [51]:
!cd /content/prod/tfjs/ && python -m http.server

Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
127.0.0.1 - - [23/May/2019 15:43:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2019 15:43:40] "GET /shoe.png HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2019 15:43:40] "GET /main.js HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2019 15:43:40] "GET /model.json HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2019 15:43:41] code 404, message File not found
127.0.0.1 - - [23/May/2019 15:43:41] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/May/2019 15:43:42] "GET /group1-shard2of3.bin HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2019 15:43:42] "GET /group1-shard1of3.bin HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2019 15:43:42] "GET /group1-shard3of3.bin HTTP/1.1" 200 -

Keyboard interrupt received, exiting.
^C


### Links


*  https://www.tensorflow.org/js/tutorials/conversion/import_keras
*  https://github.com/tensorflow/tfjs-examples/blob/master/mobilenet/index.js

